Mainly serve as a template for future strategies. The template use simple moving average (SMA) of different timeframes to generate buy and sell signals. The higher timeframe SMA is used to generate the trend, and the lower timeframe SMA is used to generate the entry and exit points.

In [1]:
from typing_extensions import Literal
import pandas as pd
import numpy as np
import talib as ta
# --- Our Imports ---
from StrategyLab.Utils.DataManager import DataManager
from StrategyLab.Config.BacktestConfig import BacktestConfig
from StrategyLab.Backtester.BacktestRunner import BacktestRunner

In [2]:
dm = DataManager(name="bitget")
ohlcv_data_s = dm.from_local("ADA/USDT:USDT", "15m", "2024-12-01")
ohlcv_data_l = dm.from_local("ADA/USDT:USDT", "4h", "2025-01-01")

Loading data from d:\ComputerScience\Trading\Quant2\StrategyLab\data\bitget\ADA_USDT_USDT_15m.csv...
Load completed successfully. Data shape: (16960, 6)
Loading data from d:\ComputerScience\Trading\Quant2\StrategyLab\data\bitget\ADA_USDT_USDT_4h.csv...
Load completed successfully. Data shape: (874, 5)


d:\ComputerScience\Trading\Quant2\StrategyLab\Utils\DataManager.py:291: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data.index = pd.to_datetime(data.index)


In [3]:
def pre_process_data(df1: pd.DataFrame, df2: pd.DataFrame):
    df1["datetime"] = df1.index
    df1.reset_index(drop=True, inplace=True)
    df2["datetime"] = df2.index
    df2.reset_index(drop=True, inplace=True)
    print("Data preprocessed.")
    return df1, df2

In [4]:
def compute_indicators_df1(
    df1: pd.DataFrame,
    period_df1: int,
):
    df1["MA"] = ta.SMA(df1["close"], timeperiod=period_df1)
    df1["STDDEV"] = ta.STDDEV(df1["close"], timeperiod=14)
    df1["ATR"] = ta.ATR(df1["high"], df1["low"], df1["close"], timeperiod=14)
    df1["SD_ATR_Spread"] = df1["STDDEV"] - df1["ATR"]
    print("Indicators computed (df1).")
    return df1


def compute_indicators_df2(
    df2: pd.DataFrame,
    fastperiod: int = 12,
    slowperiod: int = 26,
    signalperiod: int = 9,
):
    # Compute MACD
    df2["MACD"] = ta.MACD(
        df2["close"],
        fastperiod=fastperiod,
        slowperiod=slowperiod,
        signalperiod=signalperiod,
    )
    df2["MACD_signal"] = ta.MACD(
        df2["close"],
        fastperiod=fastperiod,
        slowperiod=slowperiod,
        signalperiod=signalperiod,
    )
    df2["MACD_hist"] = df2["MACD"] - df2["MACD_signal"]
    print("Indicators computed (df2).")
    return df2


def merge_htf_into_ltf(df_ltf, df_htf, suffix):
    df_ltf = df_ltf.copy()
    df_htf = df_htf.copy()

    df_ltf["datetime"] = pd.to_datetime(df_ltf["datetime"])
    df_htf["datetime"] = pd.to_datetime(df_htf["datetime"])

    df_ltf.set_index("datetime", inplace=True)
    df_htf.set_index("datetime", inplace=True)

    df_htf = df_htf.shift(1)

    df_htf_resampled = df_htf.reindex(df_ltf.index, method="ffill")
    df_merged = df_ltf.join(df_htf_resampled, rsuffix=suffix)
    df_merged.dropna(inplace=True)
    print("Starting time before merge:", df_ltf.index[0])
    print("Starting time after merge:", df_merged.index[0])
    return df_merged.reset_index()

In [5]:
df1, df2 = pre_process_data(ohlcv_data_s, ohlcv_data_l)
df1 = compute_indicators_df1(df1, 55)
df2 = compute_indicators_df2(df2, 3)

Data preprocessed.
Indicators computed (df1).
Indicators computed (df2).


In [6]:
merge_df = merge_htf_into_ltf(df1, df2, suffix="_4h")


Starting time before merge: 2024-12-01 00:00:00
Starting time after merge: 2025-01-03 12:00:00


In [7]:
merge_df.columns

Index(['datetime', 'instrument', 'open', 'high', 'low', 'close', 'volume',
       'MA', 'STDDEV', 'ATR', 'SD_ATR_Spread', 'open_4h', 'high_4h', 'low_4h',
       'close_4h', 'volume_4h', 'MA_4h', 'STDDEV_4h', 'ATR_4h',
       'SD_ATR_Spread_4h', 'trend'],
      dtype='object')

In [8]:
def compute_signals(merged_df: pd.DataFrame):
    merged_df["signal"] = 0
    # Convert numpy array to pandas Series before shifting

    # Create signal array
    signal = np.where(
        (merged_df["trend"] == 1)
        & (merged_df["close"] > merged_df["MA"])
        & (merged_df["SD_ATR_Spread"] > 0),
        1,
        np.where(
            (merged_df["trend"] == -1)
            & (merged_df["close"] < merged_df["MA"])
            & (merged_df["SD_ATR_Spread"] > 0),
            -1,
            0,
        ),
    )
    # Convert signal array to pandas Series before shifting
    merged_df["signal"] = pd.Series(signal, index=merged_df.index).shift(1).fillna(0)
    print("Signal computed.")
    return merged_df

In [9]:
solution = compute_signals(merge_df)

Signal computed.


# Set datetime to index to use the backtest

In [10]:
solution.set_index('datetime', inplace=True)

# Select Position Sizing Method

In [11]:
config = BacktestConfig(
    initial_balance=100.0,
    trading_mode="Cross",
    leverage=100,
    position_sizing_method="PercentBalance",
    position_size_pct=0.1,
    exit_on_signal_0=True,
    atr_period=14,
    atr_multiplier=2.5,
    stop_loss_pct=0.02,
    take_profit_pct=0.02,
)

In [12]:
runner = BacktestRunner(config=config, data=solution)
analysis = runner.run()
analysis.print_metrics()

Starting backtest...
Backtest finished.

--- Backtest Results ---
Period: [2025-01-03] -> [2025-05-26]
Initial Balance:        100.00
Final Balance:          0.00
ROI:                    -100.00%
Profit:                 95.22
Loss:                   195.22
Sharpe Ratio:           -6.78
Sortino Ratio:          -9.38
Calmar Ratio:           -1.00
Max Drawdown:           100.00%
Total Trades:           642
Total Good Trades:      250
Total Bad Trades:       392
Avg PnL Good Trades:    87.72%
Avg PnL Bad Trades:     -81.89%
Win Rate:               38.94%
Loss Rate:              61.06%
Profit Factor:          0.49


In [13]:
ex = analysis.results_df

# Select All Position Sizing Method

In [27]:
base_config = BacktestConfig(
    initial_balance=100.0,
    trading_mode="Cross",
    leverage=50,
    exit_on_signal_0=True,
    atr_period=14,
    atr_multiplier=2.5,
    stop_loss_pct=0.02,
    take_profit_pct=0.02,
    risk_per_trade_pct=0.02,  # For AtrVolatility
    position_size_pct=0.1,  # For PercentBalance
    position_size_fixed_amount=10.0,  # For FixedAmount ($2 margin per trade)
)

In [28]:
import StrategyLab.Backtester.ComparativeRunner as cr

In [ ]:
res = cr.run_comparative_analysis(base_config, solution)

In [ ]:
cr.print_comparison_report(res)